在程序运行过程中，总会遇到各种各样的错误。

有的错误是程序编写有问题造成的，称之为bug，bug是必须修复的。

有的错误是完全无法在程序运行过程中预测的，这类错误称之为异常。在程序中通常是必须处理的，否则，程序会因为各种问题终止并退出。

Python内置了一套异常处理机制，来帮助我们进行错误处理。还需要跟踪程序的执行，查看变量的值是否正确，这个过程称为调试。Python的pdb可以让我们以单步方式执行代码。

最后是测试，良好的测试就可以在程序修改后反复运行，确保程序输出符合我们编写的的测试。 

# 一、 错误处理

在程序运行的过程中，如果发生了错误，可以事先约定返回一个错误代码，就可以知道是否有错，以及出错的原因。
try...except..finally..错误机制

## 1.1 try机制

In [1]:
try:
    print('try ...')
    r =10 / 0 # 产生错误
    print('result:', r) # 此句不会被执行-

    except ZeroDivisionError as e: # except捕获到ZeroDivisionError,所以执行
    print('except:', e)
finally: # 执行
    print('finally ...')
print('END') # 程序流程结束

SyntaxError: invalid syntax (<ipython-input-1-630c40ca6b13>, line 6)

当我们认为某些代码可能会出错时，就用try来运行代码，如果执行出错，则后续代码不会继续执行
而是直接跳转至错误处理代码，即except语句块，执行完except后，如果有finally语句块，则执行

In [2]:
try:
    print('try ...')
    r =10 / 2 # 不产生错误
    print('result:', r) # 执行
except ZeroDivisionError as e: # 不执行
    print('except:', e)
finally: # 执行
    print('finally ...')
print('END') # 程序流程结束

try ...
result: 5.0
finally ...
END


没有错误发生，所以except语句块不会被执行，如果有 finally，则一定会被执行。

错误种类很多，如果发生了不同类型的错误，应该有不同的except语句块处理。

In [3]:
try:
    print('try ...')
    r = 10 / int('a')
    print('result:', r)
except ValueError as e:
    print('ValueError:', e)
except ZeroDivisionError as  e:
    print('ZeroDivisionError:', e)
finally:
    print('finally ...')
print('END')

try ...
ValueError: invalid literal for int() with base 10: 'a'
finally ...
END


In [4]:
# try...except可以跨越多层调用

In [5]:
# main()调用foo(),foo()调用bar(),结果bar()出错，只要main()捕获到即可

In [6]:
def foo(s):
    return 10 / int(s)

def bar(s):
    return foo(s) * 2

def main():
    try:
        bar('0')
    except Exception as e:
        print('Error:', e)
    finally:
        print('finally...')

## 1.2 调用栈

如果错误没有被捕获，就一直往上抛，最后被Python解释器捕获，打印一个错误信息，然后程序退出

In [7]:
def foo(s):
    return 10 / int(s)

def bar(s):
    return foo(s) * 2

def main():
    bar('0')

In [8]:
main()

ZeroDivisionError: division by zero

In [ ]:
# 解读错误

# Traceback (most recent call last)
# 错误的跟踪信息

# <ipython-input-13-263240bbee7e> in <module>
#----> 1 main()
# 调用main()出错

# <ipython-input-12-eefab5c8f7e8> in main()
#      6 
#      7 def main():
#----> 8     bar('0')
# 调用bar('0')出错

# <ipython-input-12-eefab5c8f7e8> in bar(s)
#      3 
#      4 def bar(s):
#----> 5     return foo(s) * 2
#      6 
#      7 def main():
# 调用return foo(s) * 2出错

#<ipython-input-12-eefab5c8f7e8> in foo(s)
#      1 def foo(s):
#----> 2     return 10 / int(s)
#      3 
#      4 def bar(s):
#      5     return foo(s) * 2
#
#ZeroDivisionError: division by zero
#根据错误类型ZeroDivisionError,判断int(s)没有出错，但是返回0，计算10/0时出错

出错的时候，一定要分析错误的调用栈信息，才能定位错误的位置。

## 1.3 记录错误

能捕获错误，就可以把错误堆栈打印出来，然后分析错误原因，同时让程序继续执行下去。

In [ ]:
# Python内置的logging模块可以非常容易地记录错误信息

In [ ]:
import logging

def foo(s):
    return 10 / int(s)

def bar(s):
    return foo(s) * 2

def main():
    try:
        bar('0')
    except Exception as e:
        logging.exception(e)
    print('END')

In [ ]:
main()

程序打印完错误信息后会继续执行，并正常退出通过配置，logging还可以把错误记录到日志文件里，方便事后排查

## 1.4 抛出错误

错误是class，捕获一个错误就是捕获到该class的一个实例。自己编写的函数也可以抛出错误。

In [ ]:
class FooError(ValueError):
    pass

def foo(s):
    n = int(s)
    if n==0:
        raise FooError('invalid value: %s' % s)
    return 10 / n

In [ ]:
foo('0')

只有必要的时候才定义我们自己的错误类型。尽量使用Python内置的错误类型。

## 小结

Python内置的try...except...finally用来处理错误十分方便。出错时，会分析错误信息并定位错误发生的代码位置。程序也可以
主动抛出错误，让那个调用者来处理相应的错误。但是，应该在文档写清楚可能会抛出哪些错误，以及错误产生的原因。

# 二、 调试

程序能一次写完并正确运行的概率很小。总会有各种bug需要修正。需要一套调试程序的手段来修复bug。

## 2.1 print()

In [ ]:
def foo(s):
    n = int(s)
    print('>>> n = %d' % n)
    return 10 / n

def main():
    foo('0')
    
main()

print()的运行结果包含很多垃圾信息，不是首选。

## 2.2 断言

凡是用print()来辅助查看的地方都可以用断言(assert)来替代

In [ ]:
def foo(s):
    n = int(s)
    assert n != 0, 'n is zero!' # 表达式n != 0应该是True，否则，根据程序运行的逻辑，后面的代码肯定会出错
    return 10 / n

def main():
    foo('0')
    
main()

如果断言失败，assert语句本身就会抛出AssertionError:

如果超出都是assert，和print()相比也差不多。不过启动Python解释器时可以用-0参数来关闭assert，关闭时，你可以把assert语句当作pass。

## 2.3 logging

logging不会抛出错误，而且可以输出到文件。

In [ ]:
import logging
logging.basicConfig(level=logging.INFO)

s = '0'
n = int(s)
logging.info('n = %d' % n)
print(10 / n)

logging的另一个好处是通过简单的配置，一条语句可以同时输出到不同的地方，比如console和文件。

## 2.4 pdb

启动Python的调试器pdb，让程序以单步方式运行，可以随时查看运行状态。

In [ ]:
# err.py

s = '0'
n = int(s)
print(10 / n)

输入命令n可以单步执行代码。任何时候都可以输入命令p 变量名来查看变量。输入命令q结束调试。

## 2.5 pdb.set_trace()

需要import pdb，在出错的地方放一个pdb.set_trace()，就可以设置一个断点

In [ ]:
# err.py
import pdb

s = '0'
n = int(s)
pdb.set_trace() # 运行到这里会自动暂停
print(10 / n)

运行代码，程序会自动在pdb.set_trace()暂停并进入pdb调试环境，可以用命令p查看变量，或者用命令c继续运行

## 2.6 IDE

比较直接地设置断点、单步执行，就需要一个支持调试功能的IDE：Visual Studio Code(安装Python插件)、PyCharm、Eclipse(安装pydev插件)

## 小结

写程序必须经过调试，虽然IDE调试方便，但是最后logging才是最实用的。

# 三、 单元测试

单元测试是用来对一个模块、一个函数或者一个类来进行正确性检验的测试工作。

如果单元测试通过，说明我们测试的函数能够正常工作。如果单元测试不通过，要么函数有bug，要么测试条件输入不正确，总之，需要修复使单元测试能够通过

测试为驱动的开发模式最大的好处就是确保一个程序模块的行为符合我们设计的测试用例

In [ ]:
# 编写一个Dict类，这个类的行为和dict一样，但是可以通过属性来访问

# mydict.py
# class Dict(dict):
    
#   def __init__(self, **kw):
#        super().__init__(**kw)
        
#    def __getattr__(self, key):
#        try:
#            return self[key]
#        except KeyError:
#            raise AttributeError(r"'Dict' object has no attribute '%s'" % key)
            
#    def __setattr__(self, key, value):
#        self[key] = value

In [ ]:
# 为了编写单元测试，需要引入Python自带的unittest模块

# mydict_test.py

# import unittest

# from mydict import Dict

# class TestDict(unittest.TestCase):

#   def test_init(self):
#      d = Dict(a=1, b='test')
#      self.assertEqual(d.a, 1)
#      self.assertEqual(d.b, 'test')
#      self.assertTrue(isinstance(d, dict))

#   def test_key(self):
#      d = Dict()
#      d['key'] = 'value'
#      self.assertEqual(d.key, 'value')

#   def test_attr(self):
#      d = Dict()
#      d.key = 'value'
#      self.assertTrue('k' in d)
#      self.assertEqual(d['key'], 'value')

#   def test_keyerror(self):
#      d = Dict()
#      with self.asserRaises(KeyError):
#          value = d['empty']

#   def test_attrerror(self):
#      d = Dict()
#      with self.asserRaises(AttributeError):
#          value = d.empty

编写单元测试时，需要编写一个测试类，从unittest.TestCase继承

以test开头的方法就是测试方法，不以test开头的方法不被认为使测试方法，测试的时候不会被执行

In [ ]:
# 每一类测试都需要编写一个test_xxx()方法。由于unittest.TestCase提供了很多内置的条件判断，所以只需要调用这些方法就可以断言输出是否是所期待的
# 最常用的断言

# self.assertEqual(abs(-1), 1) # 断言函数返回的结果与1相等

In [ ]:
# 另一种断言就是期待抛出指定类型的Error

# with self.assertRaises(KeyError):
#     value = d['empty']

# 通过d.empty访问不存在的key时，期待抛出AttributeError
# with self.assertRaises(AttributeError):
#     value = d.empty

## 3.1 运行单元测试

In [ ]:
# 编写好单元测试，既可以运行单元测试。最简单的运行方式时在mydict_test.py的最后加上两行代码

# if __name__ = '__main__':
#    unittest.main()

就可以把mydict_test.py当作正常python脚本运行

In [ ]:
# 另一种方式在命令行通过参数-m unittest直接运行单元测试

# -m unittest mydict_test

## 3.2 setUp与tearDown

假设测试需要启动数据库，就可以在setUp()中连接数据库，在tearDown()中关闭数据库，这样就不必再每个测试方法中重复相同的代码

In [ ]:
# class TestDict(unittest.TestCase):
    
#    def setUp(self):
#       print('setup ...')
        
#    def tearDown(self):
#        print('tearDown ...')

可以再次运行测试看看每个测试方调用前后是否会打印setUp和tearDown

## 小结

单元测试可以有效地测试某个程序模块的行为，是未来重构代码的信心保证

单元测试的测试用例要覆盖常用的输入组合、边界条件和异常

单元测试代码要非常简单，如果测试代码太复杂，那么测试代码本身就可能有bug

单元测试通过了并不意味着程序就没有bug，但是不通过程序肯定会bug

# 四、 文档测试

阅读官方文档，可以看到很多实例代码

In [ ]:
import re

In [ ]:
m = re.search('(?<=abc)def', 'abcdef')

In [9]:
m.group(0)

NameError: name 'm' is not defined

In [ ]:
# 如果这样写注释

def abs(n):
    '''
    Function to get absolute value of number.
    
    Example:
    
    >>>abs(1)
    1
    >>>abs(-1)
    1
    >>>abs(0)
    0
    '''
    return n if n >=0 else (-n)

Python内置的文档测试(doctest)模块可以直接提取注释中的代码并执行测试

class Dict(dict):
    '''
    Simple dic but also access as x.y style.
    
    >>>d1 = Dict()
    >>>d1['x'] = 100
    >>>d1.x
    100
    >>>d1.y = 200
    >>>d1['y']
    200
    >>>d2 = Dict(a=1, b=2 ,c='3')
    >>>d2.c
    '3'
    >>>d2['empty']
    Traceback (most recent call last):
        ...
    KeyError: 'empty'
    >>>d2.empty
    Traceback (most recent call last):
        ...
    AttributeError:'Dict' object has no attribute 'empty'
    '''
    
    def __init__(self, **kw):
        super(Dict, self).__init__(**kw)
        
    def __getattr__(self, key):
        try:
            return self[key]
        except KeyError:
            raise AttributeError(r"'Dict' object has no attribute '%s'" % key)
            
    def __setattr__(self, key, value):
        self[key] = value
        
if __name__=='__main__':
    import doctest
    doctest.testmod()

## 小结

doctest可测试也可以直接作为示例代码。通过某些文档生成工具，就可以自动把包含doctest的注释提取出来。